In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## EDA Insurance Group

In [17]:
# Import the dataset
Insurance_dataset_path= "C:/Users/Richie MOUHOUADI/Documents/DSTI/ML with Python/Mental_Health_State_Prediction/Data/Clean_Data/Insurance_group.csv"
target_path = "C:/Users/Richie MOUHOUADI/Documents/DSTI/ML with Python/Mental_Health_State_Prediction/Data/Clean_Data/target_column.csv"

Insurance = pd.read_csv(Insurance_dataset_path, sep=",", index_col=False)
target = pd.read_csv(target_path, index_col = False)
display (Insurance.head())
target.head()


,Medicaid and Medicare Insurance,No Insurance,Unknown Insurance Coverage,Medicaid Insurance,Medicaid Managed Insurance,Medicare Insurance,Private Insurance,Child Health Plus Insurance,Other Insurance
0,NO,NO,False,YES,YES,NO,NO,NO,NO
1,NO,NO,False,YES,YES,NO,NO,NO,NO
2,NO,NO,False,YES,YES,NO,NO,NO,NO
3,NO,NO,False,YES,NO,NO,NO,NO,NO
4,NO,NO,False,YES,YES,NO,NO,NO,NO


,Unnamed: 0,Mental Illness
0,0,YES
1,1,YES
2,2,YES
3,3,YES
4,4,YES


In [18]:
# new insurance group dataset (with the Target)
new_ins_grp = Insurance.copy()
new_ins_grp["Mental Illness"] = target["Mental Illness"].values
new_ins_grp.head()

,Medicaid and Medicare Insurance,No Insurance,Unknown Insurance Coverage,Medicaid Insurance,Medicaid Managed Insurance,Medicare Insurance,Private Insurance,Child Health Plus Insurance,Other Insurance,Mental Illness
0,NO,NO,False,YES,YES,NO,NO,NO,NO,YES
1,NO,NO,False,YES,YES,NO,NO,NO,NO,YES
2,NO,NO,False,YES,YES,NO,NO,NO,NO,YES
3,NO,NO,False,YES,NO,NO,NO,NO,NO,YES
4,NO,NO,False,YES,YES,NO,NO,NO,NO,YES


In [19]:
new_ins_grp.columns

Index(['Medicaid and Medicare Insurance', 'No Insurance',
       'Unknown Insurance Coverage', 'Medicaid Insurance',
       'Medicaid Managed Insurance', 'Medicare Insurance', 'Private Insurance',
       'Child Health Plus Insurance', 'Other Insurance', 'Mental Illness'],
      dtype='object')

In [20]:
# frequency_table  function
def frequency_table(data: pd.DataFrame, target_col: str, feature_cols):
    freq_tab = pd.crosstab(index = data[target_col], columns = data[feature_cols], margins = True)
    rel_table = round(freq_tab/freq_tab.loc["All"], 2)
    return freq_tab, rel_table

In [21]:
# frequency_table  function
def frequency_table(data: pd.DataFrame, target_col: str, feature_cols):
    freq_tab = pd.crosstab(index = data[target_col], columns = data[feature_cols], margins = True)
    rel_table = round(freq_tab/freq_tab.loc["All"], 2)
    return freq_tab, rel_table

In [22]:
#analysis Program Category column

for col in new_ins_grp.columns[0:-1]:
    freq, rel = frequency_table(new_ins_grp, "Mental Illness", col)
    print("Two-way frequency table")
    print(freq)
    #print("Two-way relative frequency table")
    #print(rel)
    print("------" * 15)
    print()
    print()


Two-way frequency table
Medicaid and Medicare Insurance      NO  UNKNOWN    YES     All
Mental Illness                                                 
NO                                 4996      411    170    5577
YES                              147468    12209  28863  188540
All                              152464    12620  29033  194117
------------------------------------------------------------------------------------------


Two-way frequency table
No Insurance        NO  UNKNOWN    YES     All
Mental Illness                                
NO                4966      220    391    5577
YES             173779     4166  10595  188540
All             178745     4386  10986  194117
------------------------------------------------------------------------------------------


Two-way frequency table
Unknown Insurance Coverage   False  True     All
Mental Illness                                  
NO                            5357   220    5577
YES                         184374  4166

In [23]:
Insurance_agg_feat = new_ins_grp.copy()


In [24]:
# Aggregation function definition

def has_public_insurance(row):
    #Returns 'Yes' if any of Medicaid, Medicare, or Child Health Plus = 'YES', otherwise 'No'
    if row['Medicaid Insurance'] == 'YES' or row['Medicare Insurance'] == 'YES' or row['Child Health Plus Insurance'] == 'YES':
        return 'Yes'
    return 'No'

def has_private_or_other_insurance(row):
    #Returns 'Yes' if any of Private Insurance, Other Insurance, or No Insurance = 'YES', otherwise 'No'
    if row['Private Insurance'] == 'YES' or row['Other Insurance'] == 'YES' or row['No Insurance'] == 'YES':
        return 'Yes'
    return 'No'


def confirmed_medicaid_managed(row):
    # Returns 'Yes' only if Medicaid Managed Insurance = 'YES'; all other values considered 'No'
    if row['Medicaid Managed Insurance'] == 'YES':
        return 'Yes'
    return 'No'


def insured_or_not(row):
    # Returns 'Yes' if any of the insurance columns are marked as 'YES', otherwise 'No'
    insurance_cols = [
        'Medicaid and Medicare Insurance',
        'No Insurance',
        'Medicaid Insurance',
        'Medicaid Managed Insurance',
        'Medicare Insurance',
        'Private Insurance',
        'Child Health Plus Insurance',
        'Other Insurance'
    ]
    for col in insurance_cols:
        if row[col] == 'YES':
            return 'Yes'
    return 'No'

In [25]:

#Aggregation
Insurance_agg_feat['Insured_or_Not'] = Insurance_agg_feat.apply(insured_or_not, axis=1)
Insurance_agg_feat['Has_Public_Insurance'] = Insurance_agg_feat.apply(has_public_insurance, axis=1)
Insurance_agg_feat['Has_Private_or_Other_Insurance'] = Insurance_agg_feat.apply(has_private_or_other_insurance, axis=1)
Insurance_agg_feat['Confirmed_Medicaid_Managed'] = Insurance_agg_feat.apply(confirmed_medicaid_managed, axis=1)


In [26]:
# Drop the original individual insurance columns
insurance_cols_to_drop = [
    'Medicaid and Medicare Insurance',
    'No Insurance',
    'Medicaid Insurance',
    'Medicaid Managed Insurance',
    'Medicare Insurance',
    'Private Insurance',
    'Child Health Plus Insurance',
    'Other Insurance'
]

# Drop those columns
Insurance_agg_feat.drop(columns=insurance_cols_to_drop, inplace=True)


Insurance_agg_feat.head()


,Unknown Insurance Coverage,Mental Illness,Insured_or_Not,Has_Public_Insurance,Has_Private_or_Other_Insurance,Confirmed_Medicaid_Managed
0,False,YES,Yes,Yes,No,Yes
1,False,YES,Yes,Yes,No,Yes
2,False,YES,Yes,Yes,No,Yes
3,False,YES,Yes,Yes,No,No
4,False,YES,Yes,Yes,No,Yes


In [27]:
#Let's check again the imbalace between "Yes" an "No" in the target column "Mental Illness"

for col in Insurance_agg_feat.columns[0:-1]:
    freq, rel = frequency_table(Insurance_agg_feat, "Mental Illness", col)
    print("Two-way frequency table")
    print(freq)
    #print("Two-way relative frequency table")
    #print(rel)
    print("------" * 15)
    print()
    print()

Two-way frequency table
Unknown Insurance Coverage   False  True     All
Mental Illness                                  
NO                            5357   220    5577
YES                         184374  4166  188540
All                         189731  4386  194117
------------------------------------------------------------------------------------------


Two-way frequency table
Mental Illness    NO     YES     All
Mental Illness                      
NO              5577       0    5577
YES                0  188540  188540
All             5577  188540  194117
------------------------------------------------------------------------------------------


Two-way frequency table
Insured_or_Not    No     Yes     All
Mental Illness                      
NO               323    5254    5577
YES             5544  182996  188540
All             5867  188250  194117
------------------------------------------------------------------------------------------


Two-way frequency table
Has_Public

In [28]:
# Drop those columns
Insurance_agg_feat.drop('Mental Illness', axis=1, inplace=True)


Insurance_agg_feat.head()

,Unknown Insurance Coverage,Insured_or_Not,Has_Public_Insurance,Has_Private_or_Other_Insurance,Confirmed_Medicaid_Managed
0,False,Yes,Yes,No,Yes
1,False,Yes,Yes,No,Yes
2,False,Yes,Yes,No,Yes
3,False,Yes,Yes,No,No
4,False,Yes,Yes,No,Yes
